In [1]:
import pandas as pd
import bisect
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, SGDRegressor
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv("df_ff_factors_010225.csv")
df = df.drop(columns=["Unnamed: 0","crsp_portno"])
df = df.sort_values(by='date')
df.columns

Index(['crsp_fundno', 'date', 'mth_return', 'exp_ratio', 'turn_ratio',
       'normalised_flow', 'gdp_to_debt_ratio', 'gdp_growth_rate', 'unm_rate',
       'infl_rate', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd',
       'excess_return', 'rolling_sharpe', 'mkt_return', 'rolling_alpha_3f',
       'rolling_alpha_4f', 'rolling_alpha_5f', 'shortrun_momentum'],
      dtype='object')

In [2]:
# Generate Lagged Dataset
def create_lagged_dataset(dataset, lag, target_var, id):
    lagged_dataset = dataset.copy()
    columns_list = list(lagged_dataset.columns)
    data_join = {}
    for column in columns_list:
        if column == target_var:
            data_join[column] = lagged_dataset[column]
        for n in range(1,lag+1):
            data_join[F'{column}_L{n}'] = lagged_dataset.groupby(id)[column].shift(n)
    lagged_dataset = pd.concat(data_join.values(), axis=1, ignore_index = True)
    lagged_dataset.columns = data_join.keys()
    return lagged_dataset.dropna()

# Generate Stepped Dataset for Training
## Steps is the number of months ahead that we are forecasting, e.g. step=2 is 2 months ahead.
## Note step=1 results in no change to dataset, i.e. use generated lagged variables to forecast current. 
def create_stepped_dataset(dataset, step, target_var, id):
    
    shifted_dataset = dataset.copy()
    shifted_dataset['shifted_target'] = shifted_dataset.groupby(id)[target_var].shift(-step + 1)
    
    # Drop rows where the shifted target is NaN (these occur due to the shift operation)
    shifted_dataset = shifted_dataset.dropna(subset=['shifted_target'])
    
    # Separate the features (X) and the target (y)
    X = shifted_dataset.drop(columns=[target_var, 'shifted_target'])
    y = shifted_dataset[['shifted_target']]
    y = y.rename(columns={'shifted_target':target_var})
    return X, y

x = create_lagged_dataset(df, lag = 1, target_var='rolling_sharpe', id = 'crsp_fundno')
test_step = create_stepped_dataset(x, step=1, target_var='rolling_sharpe', id = 'crsp_fundno_L1')[1]
test_step

,rolling_sharpe
43632,2.216452
43633,3.058133
13748,-0.536935
43634,1.604677
13749,0.429084
...,...
3031,1.619481
1960,0.691406
2113,0.691738
1842,0.676394


In [3]:
import numpy as np
corr = np.corrcoef(df['rolling_alpha_5f'], df['rolling_sharpe'])
corr # 0.164 --> can use!

array([[1.        , 0.16425066],
       [0.16425066, 1.        ]])

3-Factor Model: mktrf, SMB, HML
4-Factor Model: mktrf, SMB, HML, UMD
5-Factor Model: mktrf, SMB, HML, RMW, CMA

In [4]:
df_3_factor = df.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf'])
df_3_factor = df[['date','mth_return','exp_ratio','turn_ratio','mktrf', 'smb', 'hml']]
df_4_factor = df.drop(columns=['mkt_return','cma','rmw','mth_return','rf'])
df_5_factor = df.drop(columns=['mkt_return','umd','mth_return','rf'])
df_4_factor

,crsp_fundno,date,exp_ratio,turn_ratio,normalised_flow,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,infl_rate,mktrf,smb,hml,umd,excess_return,rolling_sharpe,rolling_alpha_3f,rolling_alpha_4f,rolling_alpha_5f,shortrun_momentum
43631,32553.0,1993-08-31,0.0162,0.15,0.009529,64.101,3.5225,6.8,2.90,0.0371,0.0010,0.0013,0.0265,0.068910,3.248937,0.068910,0.068910,0.068910,0.012866
43632,32553.0,1993-09-30,0.0162,0.15,0.014881,64.101,3.5225,6.7,2.90,-0.0012,0.0298,-0.0031,0.0337,-0.004108,2.216452,-0.004108,-0.004108,-0.004108,-0.009113
43633,32553.0,1993-10-29,0.0162,0.15,0.030641,64.669,3.5225,6.8,2.90,0.0141,0.0189,-0.0276,-0.0272,0.040962,3.058133,0.040962,0.040962,0.040962,0.000383
13747,12051.0,1993-10-29,0.0188,0.77,0.070030,64.669,3.5225,6.8,2.90,0.0141,0.0189,-0.0276,-0.0272,0.034690,5.744340,0.034690,0.034690,0.034690,0.013103
13748,12051.0,1993-11-30,0.0188,0.77,0.000259,64.669,3.5225,6.6,2.90,-0.0189,-0.0127,-0.0074,-0.0474,-0.075999,-0.536935,-0.075999,-0.075999,-0.075999,-0.026033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,4610.0,2024-06-28,0.0217,0.23,-0.650182,120.040,2.5427,4.1,3.35,0.0070,-0.0568,-0.0247,0.0508,0.078518,1.619481,0.001428,0.002329,0.003201,-0.025985
1960,4330.0,2024-07-31,0.0077,0.31,-0.022800,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,0.0508,0.028784,0.691406,0.004765,0.005643,0.010978,-0.024241
2113,4333.0,2024-07-31,0.0076,0.31,-0.402964,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,0.0508,0.028779,0.691738,0.004733,0.005601,0.010953,-0.024228
1842,4327.0,2024-07-31,0.0102,0.31,-0.032107,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,0.0508,0.028569,0.676394,0.004506,0.005384,0.010697,-0.024451


In [3]:
def demeaning_fund_char(dataset, id, characteristic):
    for col in characteristic:
        dataset[f'demeaned_{col}'] = dataset[col] - dataset.groupby(id)[col].transform('mean')
    return dataset

char_to_demean = ["exp_ratio", "turn_ratio", "normalised_flow", "shortrun_momentum"]
df = demeaning_fund_char(df, id="crsp_fundno", characteristic = char_to_demean)
df = df.drop(columns = char_to_demean)   # drop original columns
df.head()

,crsp_fundno,date,mth_return,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,infl_rate,mktrf,smb,hml,...,excess_return,rolling_sharpe,mkt_return,rolling_alpha_3f,rolling_alpha_4f,rolling_alpha_5f,demeaned_exp_ratio,demeaned_turn_ratio,demeaned_normalised_flow,demeaned_shortrun_momentum
43631,32553.0,1993-08-31,0.068935,64.101,3.5225,6.8,2.9,0.0371,0.0010,0.0013,...,0.068910,3.248937,0.056070,0.068910,0.068910,0.068910,0.000025,-0.464384,0.001507,0.020627
43632,32553.0,1993-09-30,-0.004082,64.101,3.5225,6.7,2.9,-0.0012,0.0298,-0.0031,...,-0.004108,2.216452,0.027009,-0.004108,-0.004108,-0.004108,0.000025,-0.464384,0.006859,-0.001352
43633,32553.0,1993-10-29,0.040984,64.669,3.5225,6.8,2.9,0.0141,0.0189,-0.0276,...,0.040962,3.058133,0.021609,0.040962,0.040962,0.040962,0.000025,-0.464384,0.022620,0.008144
13747,12051.0,1993-10-29,0.034712,64.669,3.5225,6.8,2.9,0.0141,0.0189,-0.0276,...,0.034690,5.744340,0.021609,0.034690,0.034690,0.034690,0.007662,-0.699757,0.066601,0.016976
13748,12051.0,1993-11-30,-0.075974,64.669,3.5225,6.6,2.9,-0.0189,-0.0127,-0.0074,...,-0.075999,-0.536935,-0.010806,-0.075999,-0.075999,-0.075999,0.007662,-0.699757,-0.003170,-0.022160


In [4]:
def create_naive(dataset, step, target_var):
    naive = dataset.copy()
    naive[target_var + '_naive'] = naive.groupby('crsp_fundno')[target_var].shift(step)
    return naive

# Generates next date
def generate_next_date(list_of_dates, date):
    return list_of_dates[list_of_dates > date].min()


def process_factor_model(X_factor, y_factor, train_end, test_date):
    X_train = X_factor[X_factor['date_L1'] <= train_end].drop(columns='date_L1')
    X_test = X_factor[X_factor['date_L1'] == test_date].drop(columns='date_L1')

    y_train = y_factor.loc[X_train.index]
    y_test = y_factor.loc[X_test.index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

# create_naive(df, step = 1, target_var='rolling_sharpe')

In [6]:
df

,crsp_fundno,date,mth_return,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,infl_rate,mktrf,smb,hml,...,excess_return,rolling_sharpe,mkt_return,rolling_alpha_3f,rolling_alpha_4f,rolling_alpha_5f,demeaned_exp_ratio,demeaned_turn_ratio,demeaned_normalised_flow,demeaned_shortrun_momentum
43631,32553.0,1993-08-31,0.068935,64.101,3.5225,6.8,2.90,0.0371,0.0010,0.0013,...,0.068910,3.248937,0.056070,0.068910,0.068910,0.068910,0.000025,-0.464384,0.001507,0.020627
43632,32553.0,1993-09-30,-0.004082,64.101,3.5225,6.7,2.90,-0.0012,0.0298,-0.0031,...,-0.004108,2.216452,0.027009,-0.004108,-0.004108,-0.004108,0.000025,-0.464384,0.006859,-0.001352
43633,32553.0,1993-10-29,0.040984,64.669,3.5225,6.8,2.90,0.0141,0.0189,-0.0276,...,0.040962,3.058133,0.021609,0.040962,0.040962,0.040962,0.000025,-0.464384,0.022620,0.008144
13747,12051.0,1993-10-29,0.034712,64.669,3.5225,6.8,2.90,0.0141,0.0189,-0.0276,...,0.034690,5.744340,0.021609,0.034690,0.034690,0.034690,0.007662,-0.699757,0.066601,0.016976
13748,12051.0,1993-11-30,-0.075974,64.669,3.5225,6.6,2.90,-0.0189,-0.0127,-0.0074,...,-0.075999,-0.536935,-0.010806,-0.075999,-0.075999,-0.075999,0.007662,-0.699757,-0.003170,-0.022160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,4610.0,2024-06-28,0.078565,120.040,2.5427,4.1,3.35,0.0070,-0.0568,-0.0247,...,0.078518,1.619481,0.055058,0.001428,0.002329,0.003201,-0.001211,-1.434538,-0.575797,-0.025378
1960,4330.0,2024-07-31,0.028831,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,...,0.028784,0.691406,0.055058,0.004765,0.005643,0.010978,-0.000815,-0.039661,-0.033919,-0.013655
2113,4333.0,2024-07-31,0.028826,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,...,0.028779,0.691738,0.055058,0.004733,0.005601,0.010953,-0.000737,-0.039661,-0.817815,-0.013661
1842,4327.0,2024-07-31,0.028616,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,...,0.028569,0.676394,0.055058,0.004506,0.005384,0.010697,-0.000815,-0.039661,-0.042810,-0.013657


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error


df['date'] = pd.to_datetime(df['date']) # converting to date format
dataset = df.sort_values(by='date')

# Factor Models
df_3_factor = dataset.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf','rolling_alpha_4f','rolling_alpha_5f'])
df_4_factor = dataset.drop(columns=['mkt_return','cma','rmw','mth_return','rf','rolling_alpha_3f','rolling_alpha_5f'])
df_5_factor = dataset.drop(columns=['mkt_return','umd','mth_return','rf','rolling_alpha_3f','rolling_alpha_4f'])

# Creating Lagged and Stepped Datasets
X_dataset_3f, y_dataset_3f = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_4f, y_dataset_4f = create_stepped_dataset(create_lagged_dataset(df_4_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_5f, y_dataset_5f = create_stepped_dataset(create_lagged_dataset(df_5_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')

X_dataset_3f = X_dataset_3f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_4f = X_dataset_4f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_5f = X_dataset_5f.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset_3f['date_L1'])
percentile_70 = list_of_dates.quantile(0.7)
train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
# train_end

X_train = X_dataset_3f[X_dataset_3f['date_L1']<=train_end].drop(columns='date_L1')
X_test = X_dataset_3f[X_dataset_3f['date_L1']>train_end].drop(columns='date_L1')

y_train = y_dataset_3f.loc[X_train.index]
y_test = y_dataset_3f.loc[X_test.index]
# X_train

criterion = nn.MSELoss()  # Loss function for regression
df_features_train = X_train
df_features_test = X_test

# Convert Data to Tensors
features_train_tensor = torch.tensor(df_features_train.values, dtype=torch.float32)
features_test_tensor = torch.tensor(df_features_test.values, dtype=torch.float32)

targets_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
targets_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Handle NaNs and Infs
features_train_tensor = torch.nan_to_num(features_train_tensor, nan=0.0, posinf=0.0, neginf=0.0)
features_test_tensor = torch.nan_to_num(features_test_tensor, nan=0.0, posinf=0.0, neginf=0.0)

# List of Hyperparameters
neurons_num = [[64,32], [128,64]]
epochs_list = [100, 200, 300]
lr_list = [0.005, 0.01]
iter = ['sgd']
                    
# Store the results of MSE for each combination of epochs and learning rates
best_mse = float('inf')
best_epochs = None
best_lr = None
best_iter = None
# neurons_pair=[32,16]
for iter_method in iter:
    for neurons_pair in neurons_num:
        class FundNN(nn.Module):
            def __init__(self, input_dim):
                super(FundNN, self).__init__()
                self.fc1 = nn.Linear(input_dim, neurons_pair[0])  # Hidden Layer 1
                self.fc2 = nn.Linear(neurons_pair[0], neurons_pair[1])  # Hidden Layer 2
                self.fc3 = nn.Linear(neurons_pair[1], 1)  # Output Layer (Regression)
                self.relu = nn.ReLU()
                
            def forward(self, features):
                x = self.relu(self.fc1(features))
                x = self.relu(self.fc2(x))
                x = self.fc3(x)  # Output regression prediction
                return x

        input_dim = features_train_tensor.shape[1]  # Updated to include fund IDs
        print("Input dimension:", input_dim)
        # num_epochs = 100
        # lr=0.01
        for num_epochs in epochs_list:
            for lr in lr_list:
                # Initialize model and optimizer
                model = FundNN(input_dim)  # Ensure model is re-initialized
                if iter_method == 'adam':
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                else:
                    optimizer = optim.SGD(model.parameters(), lr=lr)

                # Training loop for the current combination of epochs and learning rate
                for epoch in range(num_epochs):
                    optimizer.zero_grad()
                    predictions = model(features_train_tensor)  # Forward pass
                    loss = criterion(predictions, targets_train_tensor)  # Compute loss
                    loss.backward()  # Backpropagation
                    optimizer.step()  # Update weights
                    
                    if (epoch + 1) % 10 == 0:
                        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

                # After training, calculate the MSE on the validation or test set
                with torch.no_grad():
                    predictions = model(features_test_tensor)
                    mse = mean_squared_error(targets_test_tensor.detach().numpy().squeeze(), predictions.detach().numpy().squeeze())  # Compute MSE
                    
                    # Store the best MSE and corresponding hyperparameters
                    if mse < best_mse:
                        best_mse = mse
                        best_epochs = num_epochs
                        best_lr = lr
                        best_neuron = neurons_pair
                        best_iter = iter_method


# After the loop, print the best combination of hyperparameters
print(f"Best MSE: {best_mse:.4f} achieved with Epochs = {best_epochs} and Learning Rate = {best_lr} and Best Neuron = {best_neuron} and Iter = {best_iter}")

# Step 6: Make Predictions
# preds = model(features_tensor).detach().numpy()
# print("Predicted Returns:", preds)

Input dimension: 14


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([38144, 1, 1])) that is different to the input size (torch.Size([38144, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 3.5145
Epoch [20/100], Loss: 2.9529
Epoch [30/100], Loss: 2.2953
Epoch [40/100], Loss: 2.0239
Epoch [50/100], Loss: 2.0227
Epoch [60/100], Loss: 2.0208


KeyboardInterrupt: 

In [ ]:
# neurons_num = [[32,16],[64,32],[128,64]]
# epochs_list = [100, 200, 300]
# lr_list = [0.001, 0.0025, 0.0075, 0.01]
# neurons_pair = [32,16]
# class FundNN(nn.Module):
#         def __init__(self, input_dim):
#             super(FundNN, self).__init__()
#             self.fc1 = nn.Linear(input_dim, neurons_pair[0])  # Hidden Layer 1
#             self.fc2 = nn.Linear(neurons_pair[0], neurons_pair[1])  # Hidden Layer 2
#             self.fc3 = nn.Linear(neurons_pair[1], 1)  # Output Layer (Regression)
#             self.relu = nn.ReLU()
            
#         def forward(self, features):
#             x = self.relu(self.fc1(features))
#             x = self.relu(self.fc2(x))
#             x = self.fc3(x)  # Output regression prediction
#             return x

# input_dim = features_train_tensor.shape[1]  # Updated to include fund IDs
# print("Input dimension:", input_dim)

# model = FundNN(input_dim)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# for epoch in range(100):
#     optimizer.zero_grad()
#     predictions = model(features_train_tensor)  # Forward pass
#     loss = criterion(predictions, targets_train_tensor)  # Compute loss
#     loss.backward()  # Backpropagation
#     optimizer.step()  # Update weights
    
#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{100}], Loss: {loss.item():.4f}')
# with torch.no_grad():
#     predictions = model(features_test_tensor)
#     mse = mean_squared_error(targets_test_tensor.detach().numpy().squeeze(), predictions.detach().numpy().squeeze())  # Compute MSE

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor


df['date'] = pd.to_datetime(df['date']) # converting to date format
dataset = df.sort_values(by='date')

# Factor Models
df_3_factor = dataset.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf','rolling_alpha_4f','rolling_alpha_5f'])
df_4_factor = dataset.drop(columns=['mkt_return','cma','rmw','mth_return','rf','rolling_alpha_3f','rolling_alpha_5f'])
df_5_factor = dataset.drop(columns=['mkt_return','umd','mth_return','rf','rolling_alpha_3f','rolling_alpha_4f'])

# Creating Lagged and Stepped Datasets
X_dataset_3f, y_dataset_3f = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_4f, y_dataset_4f = create_stepped_dataset(create_lagged_dataset(df_4_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_5f, y_dataset_5f = create_stepped_dataset(create_lagged_dataset(df_5_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')

X_dataset_3f = X_dataset_3f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_4f = X_dataset_4f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_5f = X_dataset_5f.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset_3f['date_L1'])
percentile_70 = list_of_dates.quantile(0.7)
train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
# train_end

X_train = X_dataset_3f[X_dataset_3f['date_L1']<=train_end].drop(columns='date_L1')
X_test = X_dataset_3f[X_dataset_3f['date_L1']>train_end].drop(columns='date_L1')
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = y_dataset_3f.loc[X_train.index]
y_test = y_dataset_3f.loc[X_test.index]
# X_train

df_features_train = X_train_scaled
df_features_test = X_test_scaled

neurons_num = [[64,32], [128,64]]
epochs_list = [100, 200, 300]
iter = ['adam','sgd']
lr = [0.01,0.005,0.001]
lr_type_list = ['constant','adaptive']

for iter_method in iter:
    for neuron_pair in neurons_num:
        for epoch_num in epochs_list:
            for learning_r in lr:
                for lr_type in lr_type_list:
                    model = MLPRegressor(hidden_layer_sizes=(neuron_pair[0], neuron_pair[1]), activation='relu', solver=iter_method, max_iter=epoch_num, learning_rate_init=learning_r, learning_rate=lr_type, random_state=42)
                        
                    model.fit(df_features_train, y_train.values.ravel())
                    
                    y_pred_3f = model.predict(df_features_test)
                    print(f'Done for {iter_method} neuron pair {neuron_pair}, epoch num {epoch_num} and learning rate {learning_r} and type of lr {lr_type}, MSE = {mean_squared_error(y_test, y_pred_3f)}')
# X_train

Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.01 and type of lr constant, MSE = 1.279010203837982
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.01 and type of lr adaptive, MSE = 1.279010203837982


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.005 and type of lr constant, MSE = 1.602058416496341


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.005 and type of lr adaptive, MSE = 1.602058416496341


...

Done for sgd neuron pair [64, 32], epoch num 200 and learning rate 0.001 and type of lr adaptive, MSE = 1.995395316017318
Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.01 and type of lr constant, MSE = 0.7095675799237893
Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.01 and type of lr adaptive, MSE = 1.1278139219199281
Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.005 and type of lr constant, MSE = 4.78981014751881
Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.005 and type of lr adaptive, MSE = 4.251774126307922
Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.001 and type of lr constant, MSE = 2.203799047396898


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.001 and type of lr adaptive, MSE = 2.4087367002733493
Done for sgd neuron pair [128, 64], epoch num 100 and learning rate 0.01 and type of lr constant, MSE = 1.8569150716994962
Done for sgd neuron pair [128, 64], epoch num 100 and learning rate 0.01 and type of lr adaptive, MSE = 1.8564070300398814


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


ValueError: Solver produced non-finite parameter weights. The input data may contain large values and need to be preprocessed.

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor


df['date'] = pd.to_datetime(df['date']) # converting to date format
dataset = df.sort_values(by='date')

# Factor Models
df_3_factor = dataset.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf','rolling_alpha_4f','rolling_alpha_5f'])
df_4_factor = dataset.drop(columns=['mkt_return','cma','rmw','mth_return','rf','rolling_alpha_3f','rolling_alpha_5f'])
df_5_factor = dataset.drop(columns=['mkt_return','umd','mth_return','rf','rolling_alpha_3f','rolling_alpha_4f'])

# Creating Lagged and Stepped Datasets
X_dataset_3f, y_dataset_3f = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_4f, y_dataset_4f = create_stepped_dataset(create_lagged_dataset(df_4_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_5f, y_dataset_5f = create_stepped_dataset(create_lagged_dataset(df_5_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')

X_dataset_3f = X_dataset_3f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_4f = X_dataset_4f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_5f = X_dataset_5f.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset_4f['date_L1'])
percentile_70 = list_of_dates.quantile(0.7)
train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
# train_end

X_train = X_dataset_4f[X_dataset_4f['date_L1']<=train_end].drop(columns='date_L1')
X_test = X_dataset_4f[X_dataset_4f['date_L1']>train_end].drop(columns='date_L1')
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = y_dataset_4f.loc[X_train.index]
y_test = y_dataset_4f.loc[X_test.index]
# X_train

df_features_train = X_train_scaled
df_features_test = X_test_scaled

neurons_num = [[64,32], [128,64]]
epochs_list = [100, 200, 300]
iter = ['adam','sgd']
lr = [0.01,0.005,0.001]
lr_type_list = ['constant','adaptive']

for iter_method in iter:
    for neuron_pair in neurons_num:
        for epoch_num in epochs_list:
            for learning_r in lr:
                for lr_type in lr_type_list:
                    model = MLPRegressor(hidden_layer_sizes=(neuron_pair[0], neuron_pair[1]), activation='relu', solver=iter_method, max_iter=epoch_num, learning_rate_init=learning_r, learning_rate=lr_type, random_state=42)
                        
                    model.fit(df_features_train, y_train.values.ravel())
                    
                    y_pred_3f = model.predict(df_features_test)
                    print(f'Done for {iter_method} neuron pair {neuron_pair}, epoch num {epoch_num} and learning rate {learning_r} and type of lr {lr_type}, MSE = {mean_squared_error(y_test, y_pred_3f)}')
# X_train

Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.01 and type of lr constant, MSE = 1.1453430626746925
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.01 and type of lr adaptive, MSE = 1.1453430626746925
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.005 and type of lr constant, MSE = 4.524469664367805
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.005 and type of lr adaptive, MSE = 4.524469664367805


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.001 and type of lr constant, MSE = 3.5924154125308223


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.001 and type of lr adaptive, MSE = 3.5924154125308223
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.01 and type of lr constant, MSE = 1.1453430626746925
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.01 and type of lr adaptive, MSE = 1.1453430626746925
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.005 and type of lr constant, MSE = 4.524469664367805
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.005 and type of lr adaptive, MSE = 4.524469664367805
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.001 and type of lr constant, MSE = 5.790139524868471
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.001 and type of lr adaptive, MSE = 5.790139524868471
Done for adam neuron pair [64, 32], epoch num 300 and learning rate 0.01 and type of lr constant, MSE = 1.1453430626746925
Done for adam n

...

Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.005 and type of lr adaptive, MSE = 3.186112218546558
Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.001 and type of lr constant, MSE = 1.3357846293016593


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.001 and type of lr adaptive, MSE = 1.2571029280564536


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


ValueError: Solver produced non-finite parameter weights. The input data may contain large values and need to be preprocessed.

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor


df['date'] = pd.to_datetime(df['date']) # converting to date format
dataset = df.sort_values(by='date')

# Factor Models
df_3_factor = dataset.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf','rolling_alpha_4f','rolling_alpha_5f'])
df_4_factor = dataset.drop(columns=['mkt_return','cma','rmw','mth_return','rf','rolling_alpha_3f','rolling_alpha_5f'])
df_5_factor = dataset.drop(columns=['mkt_return','umd','mth_return','rf','rolling_alpha_3f','rolling_alpha_4f'])

# Creating Lagged and Stepped Datasets
X_dataset_3f, y_dataset_3f = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_4f, y_dataset_4f = create_stepped_dataset(create_lagged_dataset(df_4_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')
X_dataset_5f, y_dataset_5f = create_stepped_dataset(create_lagged_dataset(df_5_factor, lag=1,target_var='rolling_sharpe', id = 'crsp_fundno'),step=1,target_var='rolling_sharpe', id = 'crsp_fundno_L1')

X_dataset_3f = X_dataset_3f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_4f = X_dataset_4f.drop(columns=['crsp_fundno_L1'], errors='ignore')
X_dataset_5f = X_dataset_5f.drop(columns=['crsp_fundno_L1'], errors='ignore')

list_of_dates = pd.to_datetime(X_dataset_5f['date_L1'])
percentile_70 = list_of_dates.quantile(0.7)
train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
# train_end

X_train = X_dataset_5f[X_dataset_5f['date_L1']<=train_end].drop(columns='date_L1')
X_test = X_dataset_5f[X_dataset_5f['date_L1']>train_end].drop(columns='date_L1')
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = y_dataset_4f.loc[X_train.index]
y_test = y_dataset_4f.loc[X_test.index]
# X_train

df_features_train = X_train_scaled
df_features_test = X_test_scaled

neurons_num = [[64,32], [128,64]]
epochs_list = [100, 200, 300]
iter = ['adam','sgd']
lr = [0.01,0.005,0.001]
lr_type_list = ['constant','adaptive']

for iter_method in iter:
    for neuron_pair in neurons_num:
        for epoch_num in epochs_list:
            for learning_r in lr:
                for lr_type in lr_type_list:
                    model = MLPRegressor(hidden_layer_sizes=(neuron_pair[0], neuron_pair[1]), activation='relu', solver=iter_method, max_iter=epoch_num, learning_rate_init=learning_r, learning_rate=lr_type, random_state=42)
                        
                    model.fit(df_features_train, y_train.values.ravel())
                    
                    y_pred_3f = model.predict(df_features_test)
                    print(f'Done for {iter_method} neuron pair {neuron_pair}, epoch num {epoch_num} and learning rate {learning_r} and type of lr {lr_type}, MSE = {mean_squared_error(y_test, y_pred_3f)}')
# X_train

Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.01 and type of lr constant, MSE = 2.3070154263271814
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.01 and type of lr adaptive, MSE = 2.3070154263271814
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.005 and type of lr constant, MSE = 1.5089516469186992
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.005 and type of lr adaptive, MSE = 1.5089516469186992
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.001 and type of lr constant, MSE = 2.6691287756133932
Done for adam neuron pair [64, 32], epoch num 100 and learning rate 0.001 and type of lr adaptive, MSE = 2.6691287756133932
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.01 and type of lr constant, MSE = 2.3070154263271814
Done for adam neuron pair [64, 32], epoch num 200 and learning rate 0.01 and type of lr adaptive, MSE = 2.3070154263271814
Done for ada

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for adam neuron pair [128, 64], epoch num 100 and learning rate 0.001 and type of lr constant, MSE = 1.8184439023958763


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for adam neuron pair [128, 64], epoch num 100 and learning rate 0.001 and type of lr adaptive, MSE = 1.8184439023958763
Done for adam neuron pair [128, 64], epoch num 200 and learning rate 0.01 and type of lr constant, MSE = 1.1692933579470066
Done for adam neuron pair [128, 64], epoch num 200 and learning rate 0.01 and type of lr adaptive, MSE = 1.1692933579470066
Done for adam neuron pair [128, 64], epoch num 200 and learning rate 0.005 and type of lr constant, MSE = 1.1797664453948635
Done for adam neuron pair [128, 64], epoch num 200 and learning rate 0.005 and type of lr adaptive, MSE = 1.1797664453948635
Done for adam neuron pair [128, 64], epoch num 200 and learning rate 0.001 and type of lr constant, MSE = 2.2452124834702425
Done for adam neuron pair [128, 64], epoch num 200 and learning rate 0.001 and type of lr adaptive, MSE = 2.2452124834702425
Done for adam neuron pair [128, 64], epoch num 300 and learning rate 0.01 and type of lr constant, MSE = 1.1692933579470066
Don

...

Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.005 and type of lr adaptive, MSE = 1.8831616486504827
Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.001 and type of lr constant, MSE = 2.382270910375781


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Done for sgd neuron pair [64, 32], epoch num 300 and learning rate 0.001 and type of lr adaptive, MSE = 2.451359728453484


C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_base.py:172: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:205: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached

ValueError: Solver produced non-finite parameter weights. The input data may contain large values and need to be preprocessed.

In [36]:
mse = mean_squared_error(targets_test_tensor.detach().numpy().squeeze(), predictions.detach().numpy().squeeze())
mse

2.2766843

In [13]:
# Training Cycle
def overall_function(dataset, outcome):
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    # dataset = dataset.replace(np.inf, 0)
    # dataset = dataset.replace(-np.inf, 0)
    
    # Factor Models
    df_3_factor = dataset.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf','rolling_alpha_4f','rolling_alpha_5f'])
    df_4_factor = dataset.drop(columns=['mkt_return','cma','rmw','mth_return','rf','rolling_alpha_3f','rolling_alpha_5f'])
    df_5_factor = dataset.drop(columns=['mkt_return','umd','mth_return','rf','rolling_alpha_3f','rolling_alpha_4f'])
    
    # Creating Lagged and Stepped Datasets
    X_dataset_3f, y_dataset_3f = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var=outcome, id = 'crsp_fundno'),step=1,target_var=outcome, id = 'crsp_fundno_L1')
    X_dataset_4f, y_dataset_4f = create_stepped_dataset(create_lagged_dataset(df_4_factor, lag=1,target_var=outcome, id = 'crsp_fundno'),step=1,target_var=outcome, id = 'crsp_fundno_L1')
    X_dataset_5f, y_dataset_5f = create_stepped_dataset(create_lagged_dataset(df_5_factor, lag=1,target_var=outcome, id = 'crsp_fundno'),step=1,target_var=outcome, id = 'crsp_fundno_L1')
    
    X_dataset_3f = X_dataset_3f.drop(columns=['crsp_fundno_L1'], errors='ignore')
    X_dataset_4f = X_dataset_4f.drop(columns=['crsp_fundno_L1'], errors='ignore')
    X_dataset_5f = X_dataset_5f.drop(columns=['crsp_fundno_L1'], errors='ignore')

    list_of_dates = pd.to_datetime(X_dataset_3f['date_L1'])
    percentile_70 = list_of_dates.quantile(0.7)
    train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
    df_end = list_of_dates.max()

    results = []
    
    while train_end != df_end:
        
        test_date = generate_next_date(list_of_dates, train_end)
        if pd.isna(test_date):
            break 

        # Process data for modeling
        X_train_3f, X_test_3f, y_train_3f, y_test_3f = process_factor_model(X_dataset_3f, y_dataset_3f, train_end, test_date)
        X_train_4f, X_test_4f, y_train_4f, y_test_4f = process_factor_model(X_dataset_4f, y_dataset_4f, train_end, test_date)
        X_train_5f, X_test_5f, y_train_5f, y_test_5f = process_factor_model(X_dataset_5f, y_dataset_5f, train_end, test_date)

        # Model 1: Naive Model
        X_naive = X_dataset_3f[X_dataset_3f['date_L1'] == test_date][['rolling_sharpe_L1']]
        print('Naive Fitted')
        
        # Adding Naive Results
        df_in_loop = y_test_3f.copy()
        df_in_loop['naive'] = X_naive.values
        
        # Model 2: Linear Regression
        linear = LinearRegression()
        linear.fit(X_train_3f, y_train_3f.values.ravel())
        linear3fpred = linear.predict(X_test_3f)
        linear.fit(X_train_4f, y_train_4f.values.ravel())
        linear4fpred = linear.predict(X_test_4f)
        linear.fit(X_train_5f, y_train_5f.values.ravel())
        linear5fpred = linear.predict(X_test_5f)
        
        print('Linear Regression 3-Factor Fitted')
        print('Linear Regression 4-Factor Fitted')
        print('Linear Regression 5-Factor Fitted')

        ## adding
        df_in_loop['linear_3f'] = linear3fpred
        df_in_loop['linear_4f'] = linear4fpred
        df_in_loop['linear_5f'] = linear5fpred

        # Model 3: Feedforward Neural Network
        model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='logistic', solver='adam', max_iter=200, random_state=42)
        
        model.fit(X_train_3f, y_train_3f.values.ravel())
        print('FFNN 3-Factor Fitted')
        y_pred_3f = model.predict(X_test_3f)
        
        model.fit(X_train_4f, y_train_4f.values.ravel())
        print('FFNN 4-Factor Fitted')
        y_pred_4f = model.predict(X_test_4f)
        
        model.fit(X_train_5f, y_train_5f.values.ravel())
        print('FFNN 5-Factor Fitted')
        y_pred_5f = model.predict(X_test_5f)

        # Adding FFNN results
        df_in_loop['ffnn_3f'] = y_pred_3f
        df_in_loop['ffnn_4f'] = y_pred_4f
        df_in_loop['ffnn_5f'] = y_pred_5f
        
        # Model 4: Basic RNN
        
        # Model 5: Random Forest Regression
        model = RandomForestRegressor(n_estimators=100, random_state=40, n_jobs=-1)

        model.fit(X_train_3f, y_train_3f.values.ravel())
        print('RF 3-Factor Fitted')
        y_pred_3f = model.predict(X_test_3f)

        model.fit(X_train_4f, y_train_4f.values.ravel())
        print('RF 4-Factor Fitted')
        y_pred_4f = model.predict(X_test_4f)

        model.fit(X_train_5f, y_train_5f.values.ravel())
        print('RF 5-Factor Fitted')
        y_pred_5f = model.predict(X_test_5f)

        df_in_loop['rf_3f'] = y_pred_3f
        df_in_loop['rf_4f'] = y_pred_4f
        df_in_loop['rf_5f'] = y_pred_5f
        
        # Add results into loop
        results.append(df_in_loop)
        train_end = test_date
        num_remaining_dates = len(list(set(date for date in list_of_dates if date > test_date)))
        print(f'{num_remaining_dates} dates remaining')
    
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

In [14]:
y = overall_function(dataset = df, outcome = 'rolling_sharpe')
y

Naive Fitted
Linear Regression 3-Factor Fitted
Linear Regression 4-Factor Fitted
Linear Regression 5-Factor Fitted


c:\Users\Xuan Le\OneDrive - National University of Singapore\Documents\University\y4s2\DSE4101 Capstone Project in DSE I\Mutual-Funds-Forecasting\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


FFNN 3-Factor Fitted


c:\Users\Xuan Le\OneDrive - National University of Singapore\Documents\University\y4s2\DSE4101 Capstone Project in DSE I\Mutual-Funds-Forecasting\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


FFNN 4-Factor Fitted


...

c:\Users\Xuan Le\OneDrive - National University of Singapore\Documents\University\y4s2\DSE4101 Capstone Project in DSE I\Mutual-Funds-Forecasting\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


FFNN 4-Factor Fitted


c:\Users\Xuan Le\OneDrive - National University of Singapore\Documents\University\y4s2\DSE4101 Capstone Project in DSE I\Mutual-Funds-Forecasting\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


FFNN 5-Factor Fitted
RF 3-Factor Fitted
RF 4-Factor Fitted
RF 5-Factor Fitted
0 dates remaining


,rolling_sharpe,naive,linear_3f,linear_4f,linear_5f,ffnn_3f,ffnn_4f,ffnn_5f,rf_3f,rf_4f,rf_5f
0,0.075880,-0.175525,1.107035,1.293003,1.362865,-0.573290,0.328114,-0.107512,-0.198595,-0.257246,0.065282
1,0.620161,0.355801,1.394467,1.410295,1.470482,0.166323,0.725197,0.340878,0.521055,0.392970,0.486536
2,-0.198237,-0.053156,1.307230,1.311123,1.353404,-0.100094,0.303508,-0.120042,-0.003408,-0.110633,0.059762
3,-0.721097,-0.824169,1.203771,1.176778,1.309823,-0.947176,-0.468088,-1.150902,-0.818876,-0.850210,-0.815150
4,0.541852,0.341311,1.319059,1.320656,1.415899,0.312471,0.617447,0.376172,0.458013,0.425351,0.466301
...,...,...,...,...,...,...,...,...,...,...,...
16200,0.622349,0.910776,0.599828,0.723024,0.646888,0.393691,1.012086,0.547661,0.757019,0.713474,0.714885
16201,0.676394,0.622349,0.560894,0.647742,0.591270,0.681270,0.627928,0.657690,0.632270,0.573459,0.667599
16202,0.691406,0.637319,0.567638,0.654383,0.597609,0.694414,0.641716,0.669445,0.667012,0.595375,0.712852
16203,0.691738,0.637960,0.565093,0.651797,0.595029,0.694673,0.641201,0.668733,0.668703,0.609311,0.718839


Evaluation

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def mean_absolute_scaled_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    naive_mae = np.mean(np.abs(y_true[1:] - y_true[:-1]))
    mae = mean_absolute_error(y_true, y_pred)
    mase = mae / naive_mae if naive_mae != 0 else np.nan

    return mase 

def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)

    return {"Model": model_name, "MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2, "MASE": mase}

def evaluate_all_models(results):
    models = ['naive', 'linear_3f', 'linear_4f', 'linear_5f', 
              'ffnn_3f', 'ffnn_4f', 'ffnn_5f', 'rf_3f', 'rf_4f', 'rf_5f']

    metrics = []

    for model in models:
        if model in results.columns:
            metrics.append(evaluate_model(results['rolling_sharpe'], results[model], model))
    
    return pd.DataFrame(metrics)

In [16]:
metrics_results = evaluate_all_models(y)
metrics_results

,Model,MAE,MSE,RMSE,R2,MASE
0,naive,0.423464,1.824658,1.350799,-0.051243,0.647748
1,linear_3f,0.651032,0.772964,0.879184,0.554671,0.995847
2,linear_4f,0.640803,0.759496,0.871491,0.562430,0.980199
3,linear_5f,0.666076,0.803326,0.896285,0.537178,1.018859
4,ffnn_3f,0.764517,1.040084,1.019845,0.400775,1.169438
5,ffnn_4f,0.823869,1.191638,1.091622,0.313459,1.260225
6,ffnn_5f,0.806309,1.400795,1.183552,0.192958,1.233365
7,rf_3f,0.443503,0.373461,0.611115,0.784837,0.678401
8,rf_4f,0.444921,0.379784,0.616266,0.781195,0.680570
9,rf_5f,0.447120,0.392387,0.626408,0.773933,0.683934


Random Test Codes

In [222]:
# from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
# from sklearn.metrics import r2_score
# X, y = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var='mth_return'),step=1,target_var='mth_return')
# X = X.replace(np.inf, 0)
# X = X.replace(-np.inf, 0)
# # X

# list_of_dates = df['date'].unique()
# list_of_dates = pd.to_datetime(df['date']) # Converting to date format

# percentile_70 = list_of_dates.quantile(0.7)
# med_date = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
# next_date = list_of_dates[list_of_dates > med_date].min()

# X['date_L1'] = pd.to_datetime(X['date_L1'])
# X_train_3f = X[X['date_L1'] < med_date]
# y_train_3f = y[:len(X_train_3f)]
# X_test_3f = X[X['date_L1'] == next_date]
# y_test_3f = y[len(X_train_3f):(len(X_train_3f)+len(X_test_3f))]
# final = y_test_3f.copy()
# X_train_3f = X_train_3f.drop(columns='date_L1')
# X_test_3f = X_test_3f.drop(columns='date_L1')

# scaler = StandardScaler()

# X_train_3f = scaler.fit_transform(X_train_3f)
# X_test_3f = scaler.transform(X_test_3f)

# model = LinearRegression()
# model.fit(X_train_3f, y_train_3f)

# y_pred = model.predict(X_test_3f)

# model2 = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=200)
# model2.fit(X_train_3f, y_train_3f)

# y_pred2 = model2.predict(X_test_3f)

# tscv = TimeSeriesSplit(n_splits = 5)
# ridge_cv = RidgeCV(cv = tscv)
# ridge_cv.fit(X_train_3f, y_train_3f)

# model3 = Ridge(alpha = ridge_cv.alpha_)
# model3.fit(X_train_3f, y_train_3f)
# y_pred3 = model3.predict(X_test_3f)

# lasso_cv = LassoCV(cv = tscv, random_state = 18, max_iter = 100000)
# lasso_cv.fit(X_train_3f, y_train_3f)

# # Create the Lasso model with the optimal alpha value
# lasso_model = Lasso(alpha = lasso_cv.alpha_)
# lasso_model.fit(X_train_3f, y_train_3f)
# y_pred4 = lasso_model.predict(X_test_3f)

# elasticnet_cv = ElasticNetCV(cv = tscv, max_iter = 100000)
# elasticnet_cv.fit(X_train_3f, y_train_3f)
# elasticnet_model = ElasticNet(alpha = elasticnet_cv.alpha_, l1_ratio = elasticnet_cv.l1_ratio_)
# elasticnet_model.fit(X_train_3f, y_train_3f)
        
# y_pred5 = elasticnet_cv.predict(X_test_3f)

# final['linear'] = y_pred
# final['mlp'] = y_pred2
# # final['ridge'] = y_pred3
# # final['lasso'] = y_pred4
# # final['elasticnet'] = y_pred5

# final
# X_test_3f
# y_pred - y_test_3f
# sum(abs((y_pred-y_test_3f)/y_test_3f))/len(y_test_3f)

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passe

,mth_return,linear,mlp,ridge,lasso,elasticnet
10946,0.065153,0.021697,0.028095,0.021687,0.020224,0.020162
39688,0.061436,0.025942,0.028074,0.025939,0.025101,0.025072
5363,0.016949,0.016051,0.011053,0.016035,0.014860,0.014763
47379,0.060768,0.019023,0.010773,0.019012,0.018111,0.018036
28685,0.039707,0.019021,0.012601,0.019010,0.018229,0.018155
...,...,...,...,...,...,...
47060,0.062213,0.026357,0.028813,0.026354,0.025445,0.025418
35538,0.063594,0.018598,0.017622,0.018585,0.017400,0.017319
38224,0.060625,0.027637,0.031756,0.027635,0.026746,0.026727
16851,0.013965,0.025182,0.028781,0.025177,0.024109,0.024072


In [229]:
from sklearn.metrics import r2_score
r2_linear = r2_score(final['mth_return'], final['linear'])
r2_mlp = r2_score(final['mth_return'], final['mlp'])
# r2_ridge = r2_score(final['mth_return'], final['ridge'])
# r2_lasso = r2_score(final['mth_return'], final['lasso'])
# r2_elasticnet = r2_score(final['mth_return'], final['elasticnet'])
r2_mlp

-1.672832028647084